In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt

# Path to your norm_logs.jsonl file
file_path = '../norm_logs.jsonl'  # Replace with the actual file path

# Load the data from the JSONL file
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line.strip()))

# Convert the data to a numpy array
data_array = np.array(data)

# Calculate the averages for each entry across all lines
averages = np.mean(data_array, axis=0)

print(len(averages))

# Plot the averages
plt.figure(figsize=(10, 6))
plt.plot(averages, label="Average Values")
plt.title('Average Values Across All Lines')
plt.xlabel('Entry Index')
plt.ylabel('Average Value')
plt.grid(True)
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'numpy'